# Sandpiper usage

Sandpiper is a WPS server that resolves rules for climatological impacts. To get started, first instatiate the client. Here, the client will try to connect to a local or remote `sandpiper` instance, depending on whether the environment variable `WPS_URL` is defined.

In [13]:
from birdy import WPSClient
from pkg_resources import resource_filename
import json
import requests
import os

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "sandpiper":
    os.chdir('../../../')

sandpiper_url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/sandpiper/wps'
# url = os.environ.get('WPS_URL', sandpiper_url)
sandpiper = WPSClient(sandpiper_url)

The list of available processes can be diplayed using the `?` command (ex. `bird?`). Furthermore, help for individual processes can be diplayed using the same command (ex. `bird.process?`).

In [14]:
sandpiper?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x7fb8c64a54f0>
File:            /tmp/sandpiper-venv/lib/python3.8/site-packages/birdy/client/base.py
Docstring:      
A Web Processing Service for the Plan2Adapt Rule Engine.

Processes
---------

resolve_rules
    Resolve climatological impacts rules
Class docstring:
Returns a class where every public method is a WPS process available at
the given url.

Example:
    >>> emu = WPSClient(url='<server url>')
    >>> emu.hello('stranger')
    'Hello stranger'
Init docstring: 
Args:
    url (str): Link to WPS provider. config (Config): an instance
    processes: Specify a subset of processes to bind. Defaults to all
        processes.
    converters (dict): Correspondence of {mimetype: class} to convert
        this mimetype to a python object.
    username (str): passed to :class:`owslib.wps.WebProcessingService`
    password (str): passed to :class:`owslib.wps.WebProcessingService`
    headers (str): pass

In [15]:
sandpiper.resolve_rules?

Signature:
sandpiper.resolve_rules(
    csv,
    date_range='2080',
    region='bc',
    geoserver='https://docker-dev03.pcic.uvic.ca/geoserver/bc_regions/ows',
    connection_string='postgres://ce_meta_ro@db3.pcic.uvic.ca/ce_meta',
    ensemble='p2a_rules',
    loglevel='INFO',
)
Docstring:
Resolve climatological impacts rules

Parameters
----------
csv : string
    Path to CSV file
date_range : {'2020', '2050', '2080'}string
    30 year period for data
region : {'bc', 'alberni_clayoquot', 'boreal_plains', 'bulkley_nechako', 'capital', 'cariboo', 'central_coast', 'central_kootenay', 'central_okanagan', 'columbia_shuswap', ...}string
    Impacted region
geoserver : string
    Geoserver URL
connection_string : string
    Database connection string
ensemble : string
    Ensemble name filter for data files
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
json : ComplexData:mimetype:`application/json`
    JSON file
File:      ~

Now that we know which process we wish to run, we can use the docstring to ensure we provide the appropriate parameters. Below we use a `credentials.json` file to provide the full connection string to the process.

In [18]:
try:
    with open('credentials.json') as f:
        data = json.load(f)
        connection_string = data['connection_string']
except Exception as e:
    print("You must run `make credentials` to create the required file")
    raise e

if connection_string == 'not_set':
    print("You must set the `connection_string` param in the `credentials.json` in order to run this demo")

else:
    output = sandpiper.resolve_rules(
        csv='tests/data/rules_small.csv',
        connection_string=connection_string
    )

Once the process has completed we can extract the results and ensure it is what we expected.

In [25]:
rules = json.loads(requests.get(output.get()[0]).content.decode('utf-8'))
print(rules)

{'rule_snow': True, 'rule_hybrid': True, 'rule_rain': True}


In [26]:
expected_rules = {
    'rule_snow': True,
    'rule_hybrid': True,
    'rule_rain': True
}

assert rules == expected_rules